In [52]:
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc
from visual_behavior.translator.allensdk_sessions import sdk_utils
import visual_behavior.utilities as vbu
import scipy.misc
import os

from visual_behavior import database as db
import pandas as pd
import numpy as np

from tqdm import tqdm

from multiprocessing import Pool

import shutil

In [1]:
import scipy
scipy.__version__

'1.2.1'

In [5]:
%widescreen

In [2]:
manifest_path = "/home/dougo/manifest.json"
cache = bpc.from_lims(manifest=manifest_path)

In [64]:
cache.get_session_table().columns

Index(['behavior_session_id', 'ophys_experiment_id', 'project_code',
       'session_name', 'session_type', 'equipment_name', 'date_of_acquisition',
       'specimen_id', 'sex', 'age_in_days', 'full_genotype', 'reporter_line',
       'driver_line'],
      dtype='object')

In [8]:
savepath = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames'
savepath

'/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames'

In [33]:
sdk_utils.get_oeid_from_osid(799021448, cache)

799368904

In [48]:

def save_random_frames(osid, n_frames = 50, savepath='/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames'):
    print('on osid = {}'.format(osid))
    try:
        manifest_path = "/home/dougo/manifest.json"
        cache = bpc.from_lims(manifest=manifest_path)

        oeid = sdk_utils.get_oeid_from_osid(osid,cache)

        well_known_files = db.get_well_known_files(osid).set_index('name')

        sync_path = ''.join(well_known_files.loc['OphysRigSync'][['storage_directory', 'filename']].tolist())
        sync_data = vbu.get_sync_data(sync_path)

        behavior_video_path = ''.join(well_known_files.loc['RawBehaviorTrackingVideo'][['storage_directory', 'filename']].tolist())
        behavior_video = vbu.Movie(behavior_video_path, sync_timestamps=sync_data['cam1_exposure_rising'])

        session = cache.get_session_data(oeid)
        session.licks['frame'] = session.licks['time'].map(lambda t:vbu.find_nearest_index(t,sync_data['cam1_exposure_rising']))

        lick_frames = session.licks['frame']
        non_lick_frames = list(set(np.arange(len(sync_data['cam1_exposure_rising']))) - set(lick_frames))

        for lick_frame in np.random.choice(lick_frames.values,n_frames):
            for i,frame in enumerate(range(lick_frame-3,lick_frame+4)):
    #             print(i,frame)
                image = behavior_video.get_frame(frame)
                filename = 'oeid_{}__frame_{}.png'.format(oeid,frame)
                scipy.misc.imsave(os.path.join(savepath, filename), image)
        print('done with {}'.format(osid))
    except Exception as e:
        print('failed on {}\n{}'.format(osid, e))

In [49]:
osids = cache.get_session_table().index.tolist()

In [60]:
with Pool(16) as pool:
    pool.map(save_random_frames,np.random.choice(osids,20))

on osid = 929255311
on osid = 825690666
on osid = 1002956874
on osid = 895003340
on osid = 761301325
on osid = 796044280
on osid = 883352686
on osid = 882417489
on osid = 906695476
on osid = 925478114
on osid = 898849158
on osid = 855898523
on osid = 893937838
on osid = 819949602
on osid = 763145551
on osid = 1000607943
failed on 925478114
'a' cannot be empty unless no samples are taken
on osid = 917498735
failed on 796044280
'cam1_exposure_rising'
on osid = 980062339
failed on 898849158
'RawBehaviorTrackingVideo'
on osid = 912232900
failed on 917498735
'a' cannot be empty unless no samples are taken
on osid = 803223329
failed on 906695476
'a' cannot be empty unless no samples are taken
failed on 929255311
'a' cannot be empty unless no samples are taken
failed on 883352686
'a' cannot be empty unless no samples are taken
failed on 912232900
'a' cannot be empty unless no samples are taken
done with 980062339
done with 855898523
done with 1000607943
done with 895003340
done with 882417489

In [62]:
savepath='/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation'
for subset in range(20):
    main_folder = os.path.join(savepath,'unlabeled_frames')
    subset_folder = os.path.join(savepath,'unlabeled_subset_{}'.format(subset))
    if os.path.exists(subset_folder) == False:
        os.mkdir(subset_folder)
    
    fns = np.random.choice(os.listdir(main_folder),200)
    for fn in fns:
        src = os.path.join(main_folder,fn)
        dst = subset_folder
        print('moving {} to {}'.format(src,dst))
        try:
            shutil.move(src,dst)
        except Exception as e:
            print('could not move {}\n{}'.format(src,e))


moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_871155338__frame_42591.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_882519987__frame_68625.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_967007328__frame_62234.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_763411094__frame_21714.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0


moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_993594789__frame_70721.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_794389520__frame_52954.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_972693785__frame_26585.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_892801488__frame_12032.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0


moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_915141818__frame_10555.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_907587886__frame_67101.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_981178524__frame_36533.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_840157581__frame_92172.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0


moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_982903841__frame_30796.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_762161809__frame_60543.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_981178524__frame_17428.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_993864050__frame_20376.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0


moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_891994418__frame_79032.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_887357514__frame_36856.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_887357514__frame_84299.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_895422187__frame_36770.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_0


...

moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_967007328__frame_12299.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_981178524__frame_36916.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_803736273__frame_70610.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_820307518__frame_27052.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset

moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_978866896__frame_40601.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_871155338__frame_117364.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_967906375__frame_31274.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_820307518__frame_40874.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subse

moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_993594789__frame_24616.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_817267860__frame_81444.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_722370798__frame_2090.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_794389498__frame_107589.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset

moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_959337347__frame_94786.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_840157604__frame_81369.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_974374271__frame_10181.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_746425960__frame_9818.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_

moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_789356708__frame_114739.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_803736273__frame_23485.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_862023590__frame_129509.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subset_19
moving /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_frames/oeid_862023590__frame_47823.png to /allen/programs/braintv/workgroups/nc-ophys/visual_behavior/lick_detection_validation/unlabeled_subs

In [56]:
os.listdir(main_folder)

['oeid_982903841__frame_38449.png',
 'oeid_936494970__frame_107527.png',
 'oeid_981178524__frame_11559.png',
 'oeid_993594789__frame_30099.png',
 'oeid_972884638__frame_73123.png',
 'oeid_948514870__frame_11182.png',
 'oeid_968651957__frame_61310.png',
 'oeid_853962969__frame_60742.png',
 'oeid_972693785__frame_129879.png',
 'oeid_967906375__frame_14430.png',
 'oeid_944366183__frame_78620.png',
 'oeid_982903841__frame_12814.png',
 'oeid_792812544__frame_27230.png',
 'oeid_979666568__frame_28803.png',
 'oeid_898747809__frame_40241.png',
 'oeid_887357514__frame_57784.png',
 'oeid_750852080__frame_32808.png',
 'oeid_887357514__frame_107929.png',
 'oeid_840157581__frame_23156.png',
 'oeid_750536778__frame_21257.png',
 'oeid_833631914__frame_132399.png',
 'oeid_959337347__frame_99150.png',
 'oeid_936494970__frame_107542.png',
 'oeid_817267860__frame_80730.png',
 'oeid_915141818__frame_94408.png',
 'oeid_862023590__frame_63483.png',
 'oeid_792812544__frame_54249.png',
 'oeid_967039545__frame